In [1]:
import pandas as pd

In [2]:
argotario = pd.read_csv('../../../data/argumentation/argotario.tsv', sep='\t')[['Text', 'Topic', 'Intended Fallacy']]

In [3]:
argotario = argotario.rename(columns={'Text': 'argument', 'Intended Fallacy': 'fallacy type'})
argotario = argotario[~argotario['fallacy type'].isna()]

argotario['fallacy type'] = argotario['fallacy type'].replace({
'Red Herring': 'fallacy of relevance', 
'Appeal to Emotion': 'appeal to emotion',
'No Fallacy': 'no fallacy',
'Hasty Generalization': 'faulty generalization',
'Irrelevant Authority':'fallacy of credibility',
'Ad Hominem': 'ad hominem'
})



In [4]:
argotario

,argument,Topic,fallacy type
0,"No, imagine you were a teacher and everybody w...",Should cellphones be used during class?,appeal to emotion
1,"The moon is so far away, we should focus on ou...",Has anyone been on the moon?,fallacy of relevance
2,"The green party in Germany has the opinion, th...",Is it justified to develop nuclear energy for ...,fallacy of credibility
3,"No, not at all. Nowadays kids spend all time p...",Do social networking sites have a positive imp...,fallacy of relevance
4,"Yes, whoever drinks and drives has a bad reac...",Should driving under the influence of alcohol ...,no fallacy
...,...,...,...
1339,"Sure, let's make America great again! Go ask o...",Is there a good reason for the American war on...,appeal to emotion
1340,"No. Greece is one of the countries, that, in i...",The greek exit from the EU is a necessary step.,no fallacy
1341,Interestingly the boys I know from single sex ...,Are single sex schools more effective than co-...,no fallacy
1342,"It has, earlier the people talked more with ea...",Has television destroyed the communication amo...,no fallacy


In [5]:
from Evaluate import Eval

In [6]:
evaluator_model_name = 'google/t5-v1_1-small'
ckpt_path = '../../../runs/train/model/ckp_25000.pth'
evaluator = Eval(
    ckpt_path=ckpt_path, 
    score_model_name=evaluator_model_name, 
    classify_model_name='../../../models/electra-base-mnli-M2/')  

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [35]:
scores = []
fallacies = []
for i, item in argotario.iterrows():
    sentence = item['argument']
    score = evaluator.score(sentence)
    pred = evaluator.classify_as_fallacy(sentence)
    scores.append(score.item())
    fallacies.append(item['fallacy type'])
    #print(f"TOPIC: {item['Topic']}\nArgument: {sentence}\nScore: {score}\nPredicted Fallacy: {pred}\nActual Fallacy: {item['fallacy type']}\n===================================")
  
import numpy as np
fallacies = np.array(fallacies)
scores = np.array(scores)

In [44]:
(fallacies=='no fallacy')

array([False, False, False, ...,  True,  True,  True])

In [64]:
idx = np.where(fallacies!='no fallacy')[0]
import pandas as pd
pd.DataFrame(scores[idx]).describe()

,0
count,909.000000
mean,0.316970
std,0.421411
min,0.001639
25%,0.002873
50%,0.020316
75%,0.846987
max,0.998820


In [38]:
for score, fallacy_type in zip(scores, fallacies):
    if score > 0.8:
        print(score, fallacy_type)

0.8862223029136658 fallacy of relevance
0.9425179958343506 fallacy of credibility
0.9346550703048706 no fallacy
0.9849717020988464 no fallacy
0.9982337951660156 no fallacy
0.9933213591575623 appeal to emotion
0.9981665015220642 no fallacy
0.997925877571106 no fallacy
0.9816919565200806 no fallacy
0.996658205986023 fallacy of credibility
0.8445025682449341 fallacy of relevance
0.9976955056190491 ad hominem
0.9984555244445801 no fallacy
0.9888090491294861 faulty generalization
0.9980050921440125 no fallacy
0.9711190462112427 no fallacy
0.9670609831809998 fallacy of credibility
0.9115841388702393 fallacy of relevance
0.9859069585800171 fallacy of credibility
0.9943669438362122 fallacy of relevance
0.9975976347923279 appeal to emotion
0.9983421564102173 ad hominem
0.9869343042373657 no fallacy
0.9308218955993652 fallacy of relevance
0.9962059259414673 ad hominem
0.99814772605896 no fallacy
0.9590319991111755 fallacy of relevance
0.9897964596748352 fallacy of credibility
0.9980181455612183 